In [1]:
import json
import pandas as pd
import elasticsearch
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
import os
os.getcwd()

'D:\\Projects\\AI-Restaurent-Chat-bot\\notebooks'

In [3]:
# change path for base folder
base_folder = 'D:/Projects/AI-Restaurent-Chat-bot/'
input_data_folder = base_folder+'input_data/'

In [4]:
# openai api
from openai import OpenAI
client = OpenAI()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [12]:
# Read the Excel file
df = pd.read_csv(input_data_folder+"cakes_data.csv")
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [13]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [14]:
# Convert the DataFrame to a list of dictionaries (JSON-like structure)
data = df.to_dict(orient='records')

In [15]:
# Save to a JSON file for indexing
with open("menu_data.json", "w") as f:
    json.dump(data, f, indent=4)

In [16]:
data[1]

{'id': 520,
 'name': 'apple pie cake',
 'minutes': 40,
 'tags': "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'north-american', 'pies-and-tarts', 'desserts', 'fruit', 'american', 'oven', 'cakes', 'pies', 'dietary', 'apples', 'equipment', 'number-of-servings']",
 'n_steps': 4,
 'steps': "['cream butter , sugar and the egg', 'mix together the flour , baking powder , cinnamon , salt , nutmeg , vanilla , apples , pecans , and raisins', 'mix together well with the creamed mixture', 'pour the batter into a round pie dish and bake at 350 30 to 40 minutes or until the cake is done and a toothpick inserted near the center comes out clean , not doughy']",
 'ingredients': "['sugar', 'egg', 'salt', 'butter', 'flour', 'baking powder', 'cinnamon', 'nutmeg', 'vanilla', 'baking apples', 'pecans', 'golden raisin']",
 'n_ingredients': 12,
 'user_id': 22504,
 'rating': 3,
 'review': "The flavor of this was good, but the making of it didn't go very well.  T

In [18]:
import wget
url= "https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py"
#wget.download(url)



In [19]:
import minsearch

In [20]:
index = minsearch.Index(
    text_fields = ['id', 'name', 'minutes', 'tags', 'n_steps', 'steps', 'ingredients', 'user_id', 'rating', 'review', 'rating_category',
       'calories', 'total_fat_(pdv)', 'sugar_(pdv)', 'sodium_(pdv)',
       'protein_(pdv)', 'saturated_fat_(pdv)', 'carbohydrates_(pdv)',
       'calorie_status', 'price'],
    keyword_fields=[]
)

In [21]:
data[1]

{'id': 520,
 'name': 'apple pie cake',
 'minutes': 40,
 'tags': "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'north-american', 'pies-and-tarts', 'desserts', 'fruit', 'american', 'oven', 'cakes', 'pies', 'dietary', 'apples', 'equipment', 'number-of-servings']",
 'n_steps': 4,
 'steps': "['cream butter , sugar and the egg', 'mix together the flour , baking powder , cinnamon , salt , nutmeg , vanilla , apples , pecans , and raisins', 'mix together well with the creamed mixture', 'pour the batter into a round pie dish and bake at 350 30 to 40 minutes or until the cake is done and a toothpick inserted near the center comes out clean , not doughy']",
 'ingredients': "['sugar', 'egg', 'salt', 'butter', 'flour', 'baking powder', 'cinnamon', 'nutmeg', 'vanilla', 'baking apples', 'pecans', 'golden raisin']",
 'n_ingredients': 12,
 'user_id': 22504,
 'rating': 3,
 'review': "The flavor of this was good, but the making of it didn't go very well.  T

In [22]:
for field in index.text_fields:
    texts = [doc.get(field, '') for doc in data]
    print(f"Field: {field}, Sample Texts: {texts[:5]}")  # Print first 5 texts for each field

Field: id, Sample Texts: [142, 520, 822, 916, 1633]
Field: name, Sample Texts: ['almond fudge banana cake', 'apple pie cake', 'heloise s cake mix cookies', 'momma s fair funnel cake', 'mc donald s pancakes']
Field: minutes, Sample Texts: [110, 40, 8, 20, 20]
Field: tags, Sample Texts: ["['weeknight', 'time-to-make', 'course', 'preparation', 'for-large-groups', 'desserts', 'oven', 'cakes', 'dietary', 'equipment', 'number-of-servings', '4-hours-or-less']", "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'north-american', 'pies-and-tarts', 'desserts', 'fruit', 'american', 'oven', 'cakes', 'pies', 'dietary', 'apples', 'equipment', 'number-of-servings']", "['15-minutes-or-less', 'time-to-make', 'course', 'preparation', '5-ingredients-or-less', 'drop-cookies', 'desserts', 'oven', 'easy', 'cakes', 'cookies-and-brownies', 'dietary', 'equipment', 'number-of-servings']", "['30-minutes-or-less', 'time-to-make', 'course', 'preparation', 'healthy', 'de

In [46]:
# Convert all fields in the data to strings
for doc in data:
    for field in index.text_fields:
        doc[field] = str(doc.get(field, ''))

index.fit(data)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [36]:
question ="names under 15

In [ ]:
index.search(question)

In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

In [44]:

index_name = "menu-items"

# Check if index exists
if not es.indices.exists(index=index_name):
    # Create the index with the correct mapping
    response = es.indices.create(
        index=index_name,
        body={
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
                "properties": {
                    "category": {"type": "text"},
                    "dishes": {
                        "type": "nested",
                        "properties": {
                            "dish name": {"type": "text"},
                            "documents": {
                                "type": "nested",
                                "properties": {
                                    "id": {"type": "keyword"},
                                    "question": {"type": "text"},
                                    "section": {"type": "text"},
                                    "text": {"type": "text"}
                                }
                            }
                        }
                    }
                }
            }
        }
    )
    print("Index creation response:", response)
else:
    print(f"Index '{index_name}' already exists.")


Index 'menu-items' already exists.


In [88]:

index_name = "menu-items"

# Check if index exists
if not es.indices.exists(index=index_name):
    # Create the index with the correct mapping
    response = es.indices.create(
        index=index_name,
        body={
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "dish_name": {"type": "keyword"},
            "id": {"type": "keyword"}
        }
    }
        }
    )
    print("Index creation response:", response)
else:
    print(f"Index '{index_name}' already exists.")


Index 'menu-items' already exists.


In [91]:
from tqdm.auto import tqdm
from elasticsearch.exceptions import NotFoundError, RequestError

for doc in tqdm(documents):
    try:
        es.index(index=index_name, document=doc)
    except (NotFoundError, RequestError) as e:
        print(f"Error indexing document: {doc}")
        print(e)


100%|██████████████████████████████████████████████████████████| 3584/3584 [03:17<00:00, 18.18it/s]


In [92]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
index_name = "menu-items"

# Example search query
query = {
    "query": {
        "match": {
            "documents.text": "banana"
        }
    }
}

response = es.search(index=index_name, body=query)
print(response)


{'took': 1275, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


In [93]:
response = es.search(index=index_name, body={"query": {"match_all": {}}})
print(response)

{'took': 33, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'menu-items', '_id': 'q1NncJEBM82uNR2Or_9y', '_score': 1.0, '_source': {'id': '3091.0_1', 'question': 'What are the ingredients?', 'section': 'ingredients', 'text': "The kitty litter cake contains the following ingredients: ['spice cake mix', 'eggs', 'oil', 'water', 'white cake mix', 'vanilla instant pudding mix', 'cookies', 'green food coloring', 'tootsie rolls', 'new kitty litter box', 'pooper scooper']", 'dish_name': 'kitty litter cake'}}, {'_index': 'menu-items', '_id': 'rFNncJEBM82uNR2Or_-u', '_score': 1.0, '_source': {'id': '3091.0_2', 'question': 'How many calories does it have?', 'section': 'calories', 'text': 'The kitty litter cake has 354.2 calories.', 'dish_name': 'kitty litter cake'}}, {'_index': 'menu-items', '_id': 'rVNncJEBM82uNR2Or__h', '_score': 1.0, '_source': {'id': '3091.0_3

In [46]:
# Check index mapping
mapping = es.indices.get_mapping(index=index_name)
print(mapping)

# Check index settings
settings = es.indices.get_settings(index=index_name)
print(settings)

{'menu-items': {'mappings': {'properties': {'dish_name': {'type': 'keyword'}, 'id': {'type': 'keyword'}, 'question': {'type': 'text'}, 'section': {'type': 'text'}, 'text': {'type': 'text'}}}}}
{'menu-items': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'menu-items', 'creation_date': '1724167714048', 'number_of_replicas': '0', 'uuid': 'LYmVzOZIRcqRaTQhnwAfZQ', 'version': {'created': '8040399'}}}}}


In [94]:
query = "almond fudge banana cake"

In [98]:
documents[0]

{'id': '142_1',
 'question': 'What are the ingredients?',
 'section': 'ingredients',
 'text': "The almond fudge banana cake contains the following ingredients: ['dole banana', 'sugar', 'margarine', 'eggs', 'amaretto liqueur', 'vanilla extract', 'all-purpose flour', 'unsweetened cocoa powder', 'baking soda', 'salt', 'dole almond']",
 'dish_name': 'almond fudge banana cake'}

In [116]:
def elastic_search(query, dish_name=None):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "multi_match": {
                            "query": query,
                            "fields": ["dish_name^6", "question^3", "text", "section"],
                            "type": "best_fields"
                        }
                    }
                ]
            }
        }
    }

    # Add a filter by dish_name if it's provided
    if dish_name:
        search_query["query"]["bool"]["filter"] = {
            "term": {
                "dish_name.keyword": dish_name
            }
        }

    # Perform the search
    response = es.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        doc = hit['_source']
        result_docs.append(doc)  # Collect all documents
        

    return result_docs


In [135]:
elastic_search(query)

[{'id': '3091.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The kitty litter cake contains the following ingredients: ['spice cake mix', 'eggs', 'oil', 'water', 'white cake mix', 'vanilla instant pudding mix', 'cookies', 'green food coloring', 'tootsie rolls', 'new kitty litter box', 'pooper scooper']",
  'dish_name': 'kitty litter cake'},
 {'id': '3333.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The dirt cake ii contains the following ingredients: ['oreo cookies', 'butter', 'cream cheese', 'instant vanilla pudding', 'milk', 'cool whip', 'flower pot', 'artificial flower']",
  'dish_name': 'dirt cake ii'},
 {'id': '822.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The heloise s cake mix cookies contains the following ingredients: ['cake mix', 'vegetable oil', 'eggs']",
  'dish_name': 'heloise s cake mix cookies'},
 {'id': '520.0_1',
  'question': 'What are the 

In [132]:
def build_prompt(query, search_results):
    prompt_template = """
You're an AI assistant helping with menu queries. Answer the QUESTION based on the CONTEXT provided.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context += f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [133]:
query

'what are ingredients in banana pudding cake contain'

In [134]:
search_results= elastic_search(query)
search_results

[{'id': '3091.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The kitty litter cake contains the following ingredients: ['spice cake mix', 'eggs', 'oil', 'water', 'white cake mix', 'vanilla instant pudding mix', 'cookies', 'green food coloring', 'tootsie rolls', 'new kitty litter box', 'pooper scooper']",
  'dish_name': 'kitty litter cake'},
 {'id': '3333.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The dirt cake ii contains the following ingredients: ['oreo cookies', 'butter', 'cream cheese', 'instant vanilla pudding', 'milk', 'cool whip', 'flower pot', 'artificial flower']",
  'dish_name': 'dirt cake ii'},
 {'id': '822.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The heloise s cake mix cookies contains the following ingredients: ['cake mix', 'vegetable oil', 'eggs']",
  'dish_name': 'heloise s cake mix cookies'},
 {'id': '520.0_1',
  'question': 'What are the 

In [119]:
# if using OPENAI 
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [120]:
# iF USING groq
"""
from groq import Groq   

#create client calling Groq class

client = Groq(api_key=os.getenv('GROQ_API_KEY'))

from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

"""

'\nfrom groq import Groq   \n\n#create client calling Groq class\n\nclient = Groq(api_key=os.getenv(\'GROQ_API_KEY\'))\n\nfrom openai import OpenAI\nclient = OpenAI()\n\ndef llm(prompt):\n    response = client.chat.completions.create(\n        model="llama3-8b-8192",\n        messages=[{"role": "user", "content": prompt}]\n    )\n    \n    return response.choices[0].message.content\n\n'

In [150]:
query = 'dish name please'

def rag(query):
    search_results = elastic_search(query)
    prompt=build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [151]:
rag(query)


"I'm sorry, the dish name you are asking for is baby food pineapple coconut carrot cake."